In [11]:
import pandas as pd
import time
from selenium import webdriver
from time import sleep
import datetime
from selenium.webdriver.common.by import By

In [12]:
# Create driver object. Opens browser window.
chromedriver_path = 'C:\Windows\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(chromedriver_path)

In [13]:
##Dynamic search for city,num of adults,children and rooms
city = "New York"
numOfAdults = 2
numOfChild = 0
numOfRooms = 0

In [14]:
##In this cell we define functions that each one of them scrape something else

def get_hotelName(PATH):
    name = []
    for curHotel in driver.find_elements(By.XPATH,PATH):
        try:
            name.append(curHotel.find_element(By.XPATH,'.//*[@class="uitk-heading uitk-heading-5 overflow-wrap"]').text)
        except:
            name.append(None)
    return name

def get_score(PATH):
    score = []
    for curHotel in driver.find_elements(By.XPATH,PATH):
        try:
            score.append(curHotel.find_element(By.XPATH,'.//*[@class="uitk-spacing uitk-spacing-padding-inlineend-half uitk-layout-flex-item"]').text)
        except:
            score.append(None)
    return score

def get_rating(PATH):
    rating = []
    for curHotel in driver.find_elements(By.XPATH,PATH):
        try:
            rating.append(curHotel.find_elements(By.XPATH,'.//*[@class="uitk-text uitk-type-300 uitk-text-default-theme"]')[1].text)
        except:
            rating.append(None)
    return rating

def get_review(PATH):
    review = []
    for curHotel in driver.find_elements(By.XPATH,PATH):
        try:
            review.append(curHotel.find_elements(By.XPATH,'.//*[@class="uitk-text uitk-type-300 uitk-text-default-theme"]')[2].text)
        except:
            review.append(curHotel.find_elements(By.XPATH,'.//*[@class="uitk-text uitk-type-300 uitk-text-default-theme"]')[1].text)
    return review

def get_price(PATH):
    price = []
    for curHotel in driver.find_elements(By.XPATH,PATH):
        try:
            price.append(curHotel.find_element(By.XPATH,'.//*[@class="uitk-price-subtext"]').text)
        except:
            price.append(None)
    return price

def get_district(PATH):
    district = []
    for curHotel in driver.find_elements(By.XPATH,PATH):
        try:
            district.append(curHotel.find_element(By.XPATH,'.//*[@class="uitk-text uitk-text-spacing-half truncate-lines-2 uitk-type-300 uitk-text-default-theme"]').text)
        except:
            district.append(None)
    return district

def get_includes(PATH):
    includes = []
    for curHotel in driver.find_elements(By.XPATH,PATH):
        try:
            includes.append(curHotel.find_element(By.XPATH,'.//*[@class="uitk-text truncate uitk-type-200 uitk-text-default-theme"]').text)
        except:
            includes.append(None)
    return includes

def get_refund(PATH):
    refund = []
    for curHotel in driver.find_elements(By.XPATH,PATH):
        try:
            refund.append(curHotel.find_element(By.XPATH,'.//*[@class="uitk-text uitk-type-300 uitk-text-positive-theme"]').text)
        except:
            refund.append('0')
    return refund

def get_weekend(start_date,end_date):
    date_range = pd.date_range(start_date, end_date - pd.Timedelta(days=1), freq='D')
    weekends = (date_range.weekday >= 5).sum()
    return weekends


In [15]:
current_date = datetime.datetime.now() 
lengthOfStay = 1 ##LOS
timeToTravel = 2 ##TTT
appended_data = [] 

for ttt in range(1, timeToTravel):
    # Iterate over LOS
    for los in range(1, lengthOfStay+1):
        
        ##Define dates
        checkin_date = current_date + datetime.timedelta(days=ttt)
        checkin_year = checkin_date.year
        checkin_month = checkin_date.month
        checkin_monthday = checkin_date.day
        checkout_date = checkin_date + datetime.timedelta(days=los)
        checkout_year = checkout_date.year
        checkout_month = checkout_date.month
        checkout_monthday = checkout_date.day
        days_until_checkin = (checkin_date - current_date).days
        url = f"https://expedia.com/Hotel-Search?destination={city}&d1={checkin_date}&d2={checkout_date}&adults={numOfAdults}&children={numOfChild}"
        driver.get(url)
        
        ##Maximize the window size and the auto scrolling to reveal 100 hotels
        driver.maximize_window()
        driver.execute_script("window.scrollTo(3,document.body.scrollHeight)")
        sleep(3)
        
        name = get_hotelName('.//*[@class="uitk-card uitk-card-roundcorner-all uitk-card-has-primary-theme"]')
        score = get_score('.//*[@class="uitk-card uitk-card-roundcorner-all uitk-card-has-primary-theme"]')
        rating = get_rating('.//*[@class="uitk-card uitk-card-roundcorner-all uitk-card-has-primary-theme"]')
        review = get_review('.//*[@class="uitk-card uitk-card-roundcorner-all uitk-card-has-primary-theme"]')
        price = get_price('.//*[@class="uitk-card uitk-card-roundcorner-all uitk-card-has-primary-theme"]')
        district = get_district('.//*[@class="uitk-card uitk-card-roundcorner-all uitk-card-has-primary-theme"]')
        includes = get_includes('.//*[@class="uitk-card uitk-card-roundcorner-all uitk-card-has-primary-theme"]')
        refund = get_refund('.//*[@class="uitk-card uitk-card-roundcorner-all uitk-card-has-primary-theme"]')
        weekend = get_weekend(checkin_date , checkout_date)
            
        appended_data.append(pd.DataFrame({
                'Name': name,
                'DateOfSearch': (f"{current_date.year}/{current_date.month}/{current_date.day}"),
                'Checkin' : (f"{checkin_year}/{checkin_month}/{checkin_monthday}"),
                'Checkout' : (f"{checkout_year}/{checkout_month}/{checkout_monthday}"),
                'NumOfWeekendDays' : weekend,
                'Nights':(checkout_date - checkin_date).days,
                'District':district,
                'Score':score,
                'Rating':rating,
                'Review':review,
                'Includes':includes,
                'Refundable':refund,
                'Price':price
                                       }))
        
        ##Refreshing driver every search due capcha 
        driver.quit()
        driver = webdriver.Chrome(chromedriver_path)
        
df = pd.concat(appended_data)
df.to_csv(f'{city} Expedia {current_date.day}.csv',index=False)
print(f"Succesfully saved {city} hotels as {city} Expedia.csv ")
        
driver.quit()

IndexError: list index out of range